<a href="https://colab.research.google.com/github/7jeon8gi/BIZ-AI-LAB/blob/master/KoBERT_12_10_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 라이브러리 로드

In [0]:
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install mxnet-cu101
!pip install pytorch-pretrained-bert pytorch-nlp
#!pip install git+https://git@github.com/huggingface/transformers.git@master

In [0]:
#from mxnet.gluon import nn, rnn
from mxnet import gluon, autograd
import gluonnlp as nlp
from mxnet import nd 
import mxnet as mx
import time
import itertools
import random
import torch
import numpy as np
import pandas as pd
from torch import nn

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange

In [0]:
from pytorch_pretrained_bert.modeling import *

In [0]:
#from transformers.modeling_bert import BertPreTrainedModel

#### KoBERT Loading 

In [0]:
bert_base , vocabulary = get_pytorch_kobert_model()
tokenizer = get_tokenizer()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
using cached model


In [0]:
# transform_text created by BIGTREE

def transform_text(data):
    ds = gluon.data.SimpleDataset([[data]])
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocabulary, lower=False) #vocabulary = gex_mxnet_kobert_model
    trans = nlp.data.BERTSentenceTransform(tok, max_seq_length=64, pair =False)
    res = list(ds.transform(trans))

    input_ids =[]
    input_mask =[]
    token_type_ids=[]

    for data in list(res[0][0]):
        if data ==1 : # which is stopping at padding token (2: CLS, 3:SEP) # 이렇게 쓸 경우 문장의 길이가 모두 달라짐. 옳은 것인가?
            break
        input_ids.append(data)
        input_mask.append(1) # not masking at all
        token_type_ids.append(0) # only one sentence. No NSP.

    input_ids = torch.LongTensor([input_ids])
    input_mask = torch.LongTensor([input_mask])
    token_type_ids = torch.LongTensor([token_type_ids])

    return input_ids, input_mask, token_type_ids 

##### Simple test

In [0]:
text= '자꾸 떨어지기만 하네 계속 가지고 있어야할까?'
input_ids, input_mask, token_type_ids = transform_text(text) # 자꾸 떨어지기만하네 계속 가지고 있어야할까?
_, pooled_output = bert_base(input_ids, input_mask, token_type_ids)
test_text = np.reshape(pooled_output[0].detach().numpy(),[1,-1])
print(test_text[0][0:10],'short cut')

[-0.00868336 -0.04926196 -0.17144394 -0.12452045 -0.9576996   0.9649978
  0.12238491  0.04631049  0.05755762  0.01517853] short cut


In [0]:
print(input_ids, input_mask, token_type_ids)
# input_ids = 토큰 고유 번호
# input_mask = 1 (마스킹 없음)
# token_type_ids = 0 (전부 첫번째 문장. 다음 문장 없음)

tensor([[   2, 3886, 5628, 1859, 5561, 6150, 4924, 5702,  984,  768, 3870, 7836,
         5591,  633,    3]]) tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]) tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


#### Data Preprocessing

In [0]:
# Upload the train file from your local drive
from google.colab import files
uploaded = files.upload()

Saving 6000all.csv to 6000all.csv


In [0]:
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv("6000all.csv", encoding="cp949")
le = LabelEncoder()
le.fit(np.unique(df["intent"].values))
df['intent'] = le.fit_transform(df.intent.values)
df.head()

,summary,intent
0,"엑사이엔씨는 어떤가요? 오를 줄 알았는데 횡보, 하락하네요.",12
1,계좌에 현금이 없는데 매수 가능한가요? 실수로 주식을 가지고 있던 현금보다 더 매수...,0
2,시간 외에 당했는데 gh신소재 월요일에 오를까요?,15
3,에이아이비트 다음주 전망 어때요?,15
4,에스에이엠피도 괜찮아 보이네요.,12


In [0]:
data=[]
tok = nlp.data.BERTSPTokenizer(tokenizer, vocabulary, lower=False)
for x in df["summary"].values:
    ds = gluon.data.SimpleDataset([[x]])
    trans = nlp.data.BERTSentenceTransform(tok, max_seq_length=64, pair=False) # False!
    val=list(ds.transform(trans))
    data.append(val[0][0]) # check

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in data:
  seq_mask = [float(i>1) for i in seq] #float True:1 False:0
  attention_masks.append(seq_mask)

In [0]:
input_ids = data
labels = le.fit_transform(df.intent.values)

# Use train_test_split to split our data into train and validation sets for training
from sklearn.model_selection import train_test_split
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

#### BERT finetuning

##### dd

In [0]:
# class BERTinitializer(nn.Module):
#     def init_weights(self, module):
#         """ Initialize the weights """
#         if isinstance(module, (nn.Linear, nn.Embedding)):
#             # Slightly different from the TF version which uses truncated_normal for initialization
#             # cf https://github.com/pytorch/pytorch/pull/5617
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#         elif isinstance(module, torch.nn.LayerNorm):
#             module.bias.data.zero_()
#             module.weight.data.fill_(1.0)
#         if isinstance(module, nn.Linear) and module.bias is not None:
#             module.bias.data.zero_()

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 config,
                 num_labels=20,
                 dropout=None,
                 ):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.num_labels = num_labels 
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size,num_labels)
       
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits

#### GPU

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

#Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs).to(torch.int64)
validation_inputs = torch.tensor(validation_inputs).to(torch.int64)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 16


# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels,)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size, )

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().req

In [0]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = BERTClassifier(bert = bert_base, config = bert_base.config, num_labels=20) # 라벨수 20개
model = model.cuda()

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
    ]

# This variable contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

t_total value of -1 results in schedule not being applied


In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

#### Training

In [0]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 30

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader

    
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

NameError: ignored

In [0]:
text='삼성전자 오늘팔까 내일팔까? 골라줘봐'
bert_base.cpu()
input_ids, input_mask, token_type_ids = transform_text(text)

_, pooled_output = bert_base(input_ids, input_mask, token_type_ids)
test_text = np.reshape(pooled_output[0].detach().numpy(),[1,-1])

model.cpu()
_, pooled_output = model.bert(input_ids)
after_text = np.reshape(pooled_output[0].detach().numpy(),[1,-1])

from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(test_text,after_text)

array([[0.99998575]], dtype=float32)

In [0]:
# Simple test after fine-tuning.

text1='대북주 지금이라도 팔아야할까?'
text2='대북주 오늘 당장 손절? 아니면 존버해?'


input_ids, input_mask, token_type_ids = transform_text(text1)
model.cpu()
_, pooled_output = model.bert(input_ids)
text11 = np.reshape(pooled_output[0].detach().numpy(),[1,-1])

input_ids, input_mask, token_type_ids = transform_text(text2)
_, pooled_output = model.bert(input_ids)
text22 = np.reshape(pooled_output[0].detach().numpy(),[1,-1])

cosine_similarity(text11,text22)

array([[0.3239141]], dtype=float32)

In [0]:
# Simple test after fine-tuning.

text1='삼성전자 현재가는 얼마야?'
text2='삼성전자 지금 가격 얼마인지 알려줘'


input_ids, input_mask, token_type_ids = transform_text(text1)
model.cpu()
_, pooled_output = model.bert(input_ids)
text11 = np.reshape(pooled_output[0].detach().numpy(),[1,-1])

input_ids, input_mask, token_type_ids = transform_text(text2)
_, pooled_output = model.bert(input_ids)
text22 = np.reshape(pooled_output[0].detach().numpy(),[1,-1])

cosine_similarity(text11,text22)

array([[0.70128596]], dtype=float32)

In [0]:
# Simple test after fine-tuning.

text1='다음주에는 코스닥 오를 것 같아?'
text2='주식 시장 최신 뉴스 불러와'


input_ids, input_mask, token_type_ids = transform_text(text1)
model.cpu()
_, pooled_output = model.bert(input_ids)
text11 = np.reshape(pooled_output[0].detach().numpy(),[1,-1])

input_ids, input_mask, token_type_ids = transform_text(text2)
_, pooled_output = model.bert(input_ids)
text22 = np.reshape(pooled_output[0].detach().numpy(),[1,-1])

cosine_similarity(text11,text22)

array([[-0.18468606]], dtype=float32)